In [ ]:
import sympy.physics.mechanics as me
import sympy as sm
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#import matplotlib
import time

from scipy.linalg import solve_continuous_are, det, expm

This creates a decorator to test functions for usage of CPU time, line by line.\
To see the results, this line: *profiler.print_stats()* must be added. 

In [ ]:
from line_profiler import LineProfiler

profiler = LineProfiler()

def profile(func):
    def inner(*args, **kwargs):
        profiler.add_function(func)
        profiler.enable_by_count()
        return func(*args, **kwargs)
    return inner

This is a **3D n body pendulum**\
I put it into a vertical position, linearize the equations of motion and try to stabilize in by
- moving the point $P_{01}$, it is attached to the horizontal plane
- adding a torque to every body. (This is needed, because I model the pendulum bodies as RigidBody, and their rotation cannot be stopped by moving around the point of attachment)

The control variables are
- $F_x, F_z$ forces applied to the point $P_0$ where the pendulum is attached to the plane.
- Torques applied to each body fixed frame

The 'suspension point' is modeled as a Particle, rest are RigidBody\
(I basically copied this program from some other program quite some time ago, and only enlarged it to 3D and rigid bodies)\
For some reason, which I did not investigate, it only works well for $n \in \{{ 1, 3}\} $. Other values give problems.

I do these calculations:
- I calculate the full system without controls
- I linearize to get the optimum feed back, and use this feed back to numerically solve the full EOMs
- I numerically solve the linearized equations, using the optimum feed back
- A linear system $\dfrac{d}{dt} \hat y = RHS \cdot \hat y$, with RHS = const., has the closed form solution $\hat y = e^{\displaystyle{RHS \cdot t}} \cdot \hat y_0$, and I use scypi's *expm(..)* function to solve it.

In [ ]:
#===============================================
n = 3           #number of pendulums (bodies)
#===============================================

start = time.time()    

N = me.ReferenceFrame('N')
P0 = me.Point('P0')
P0.set_vel(N, 0)                                      

# Generalized coordinates and velocities
q = []
u = []

A = [N]

Torque = []
for i in range(1, n+1):                                             # Data for pendulun i at at position [i-1]
    for j in ('x', 'y', 'z'):
        q.append(me.dynamicsymbols('q'+str(i) + str(j)))            #Rotation of frame i relative to N
        u.append(me.dynamicsymbols('u'+str(i) + str(j)))           
    A.append(me.ReferenceFrame('A'+ str(i)))
    Torque.append(me.dynamicsymbols('T' + str(i-1)))
        
q0x, q0z = me.dynamicsymbols('q0x q0z')                             # coordinates of attachment poi nt
u0x, u0z = me.dynamicsymbols('u0x, u0z')

g, l, m, c1, t = sm.symbols('g l m c1, t')                          # all masses, distances, time, friction are the same for all pendulums
r1 = sm.symbols('r1')                                               # distance from Pi to Dmc
iXX, iYY, iZZ = sm.symbols('iXX, iYY, iZZ')                         # moments of inertia around the center of mass, principal axes

Fx, Fz = me.dynamicsymbols('Fx, Fz')                                       #Control parameters. forces at attachment point acting on P01

P01 = P0.locatenew('P01', q0x*N.x + q0z*N.z)
P01.set_vel(N, u0x*N.x + u0z*N.z)

#Frame points, particles for each Pendulum, numbered 1, 2, ..., n
rot = [0]                                                           # rot, rot1 needed for the kinematical differential equations
rot1 = [0]
P = [P01]
DMC = [P01]                                                         # list of center of gravity of each body
BODY = [me.Particle('P01a', P01, m)]                                # listof bodies

for i in range(1, n+1):
    A[i].orient_body_fixed(A[0], (q[3*i-3], q[3*i-2], q[3*i-1]), '123')  
    rot.append(A[i].ang_vel_in(N))
    A[i].set_ang_vel(N, u[3*i-3] * A[i].x + u[3*i-2] * A[i].y + u[3*i-1] * A[i].z )
    rot1.append(A[i].ang_vel_in(N))

    Pi = P[i-1].locatenew('P' + str(i), l * A[i].y)                 # old way to determine points
    Pi.v2pt_theory(P[i-1], N, A[i])
    P.append(Pi)
    Dmci = P[i-1].locatenew('Dmc' + str(i), r1 * A[i].y)            # location of center of gravity
    Dmci.v2pt_theory(P[i-1], N, A[i])
    DMC.append(Dmci)

    I = me.inertia(A[i], iXX, iYY, iZZ)
    Bodyi = me.RigidBody('Body' + str(i), Dmci, A[i], m, (I, Dmci))
    BODY.append(Bodyi)   

Set up **Kane's equations**

In [ ]:
# Kanes forrmalism
start1 = time.time()

#Kinematic equations
kd = [(q0x.diff(t) - u0x)] + [(q0z.diff(t) - u0z)] + [me.dot(rot[i] - rot1[i], uv) for uv in A[i] for i in range(1, n+1)]

# forces, torques
FL = [(P01, -m*g*N.y + Fx*N.x + Fz*N.z)]

for i in range(1, n+1):
    FL.append((DMC[i], -m*g*N.y))
    FL.append((A[i], Torque[i-1]*A[i].y))

q_ind = [q0x, q0z] + q
u_ind = [u0x, u0z] + u

KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
print('MM DS', me.find_dynamicsymbols(MM))
print('MM FS', MM.free_symbols)
print(f'MM contains {sm.count_ops(MM):,} operations, {sm.count_ops(sm.cse(MM)):,} after cse, \n')

force = KM.forcing_full
print('force DS', me.find_dynamicsymbols(force))
print('force FS', force.free_symbols)
print(f'force contains {sm.count_ops(force):,} operations, {sm.count_ops(sm.cse(force)):,} after cse \n')
print("it took {:.3f} sec to establish Kane's equations".format(time.time() - start1))

**Lambdification**

In [ ]:
# Carthesian coordinates of the center points, needed only for plotting them
orte_Dmc = ([ [me.dot(ort.pos_from(P0), uv) for uv in N] for ort in DMC] )

qL = q_ind + u_ind
pL = [g, m, l, r1, iXX, iYY, iZZ, c1, Fx, Fz] + Torque                                     

MM_lam = sm.lambdify(qL + pL, MM, cse=True)
force_lam = sm.lambdify(qL + pL, force, cse=True)

orte_Dmc_lam = []
for ort in orte_Dmc:
    orte_Dmc_lam.append(sm.lambdify(qL + pL, ort))

**Numerical Integration**\
Normally in my simulations I explicitly set all parameters and initial conditions.\
As here I am after something else, linearization and control, I simply pre_set them.\
Of course they may be changed, just a bit more complicated.\
This is the uncontrolled motion, below I linearize around the vertical position and control it.

In [ ]:
start2 = time.time()

intervall = 5
schritte = 250
times = np.linspace(0, intervall, schritte)

#Anfangsbedingungen
#qL = [q0x, q0z] + q + [u0x, u0z] + u
#pL = [g, m, l, r1, iXX, iYY, iZZ, c1, Fx, Fz] + Torque    
                                  
pL_vals = [9.8, 1, 10, 5., 1/2, 1/3, 1/3, 0., 0., 0.] + [ 0. for _ in range(n)]

y0 = [0., 0.] + [0, 0, 0] * int(len(q)/3) + [0., 0.] + [0.1, np.random.choice((-1, 1)) * 0.1, 0.1] * int(len(u)/3)
                                            
def gradient(t, y, args):                        
    vals = np.concatenate((y, args))
    sol = np.linalg.solve(MM_lam(*vals), force_lam(*vals))
    return np.array(sol).T[0]

times = np.linspace(0., intervall, int(schritte*intervall))
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), atol=1.e-10, rtol=1.e-10) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start2):.3f} CPU - sec  ')

Plot the results

In [ ]:
bez = ['X', 'Y', 'Z']
fig,(ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

x_coords = np.zeros((len(times), n+1))
y_coords = np.zeros((len(times), n+1))
z_coords = np.zeros((len(times), n+1))

for i in range(len(times)):
    for k, ort in enumerate(orte_Dmc_lam):
        x_coords[i, k] = ort(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[0]
        y_coords[i, k] = ort(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[1]
        z_coords[i, k] = ort(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)[2]
    
for j in range(0, n+1):
    ax2.plot(times, x_coords[:,j], label='X-Koordinate von Punkt {}'.format(j))
    ax2.plot(times, y_coords[:,j], label='Y-Koordinate von Punkt {}'.format(j))
    ax2.plot(times, z_coords[:,j], label='Z-Koordinate von Punkt {}'.format(j))
ax2.set_title('Coordinates of center of gravity')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Coordinates [m]')
ax2.legend(loc='upper right');

**Linearisation**

I use sympy's method as expalined here: https://docs.sympy.org/latest/modules/physics/mechanics/linearize.html

Notes:
- the op_point dictionary *must* also include the values of the *accelerations* of the coordinates. This is not very obvious from the documentatrion, JM explained this to me.
- the controls variable must be *dynamic symbols*. This is clearly explained in the documentation. Here, they are $F_x, F_z, T_i$ and they are declared to be dynamic symbols above.

This is much faster than trying to do it 'from first principles' using jacobian operators - as I did it in my related example.


In [ ]:
start3 = time.time()
dict_eq = {i: 0. for i in qL} | {i.diff(t): 0. for i in u_ind}

F_A, F_B, input_vector = KM.linearize(A_and_B=True, op_point=dict_eq, new_method=True)
print('input_vector', input_vector)                                                                     # input vector contains the control parameters
print('F_A DS', me.find_dynamicsymbols(F_A))
print('F_A FS', F_A.free_symbols)
print(f'F_A contains {sm.count_ops(F_A):,} operations, {sm.count_ops(sm.cse(F_A)):,} after cse \n')

print('F_B', me.find_dynamicsymbols(F_B))
print('F_B FS', F_B.free_symbols)   
print(f'F_B contains {sm.count_ops(F_B):,} operations, {sm.count_ops(sm.cse(F_B)):,} after cse \n')

print(f'It took {(time.time() - start3):.3f} sec to linearize the EOMs')

The linearized sytem is: $\dfrac{d}{dt} \bar y = F_A \cdot \bar y + F_B \cdot \bar u$, where $\bar u$ is the control, and $F_A, F_B$ are the matrices obtained above.

Calculate the **optimal feed back** to stabilize the linearized system.\
Let P be the solution to the Riccati equation $Q + P \cdot F_A + F_A^T \cdot P - P \cdot B \cdot R^{-1} \cdot F_B^T \cdot P = 0$, where $Q, R$ are the weight matrices in the cost functional related to the coordinates and the control, respectively.\
Then $\hat{\overline{control}} = \left[ R^{-1} \cdot F_B^T \cdot P \right] \cdot \bar y$ ist the optimal control.\
For lack of better reason, I pick $R$ and $Q$ to be the unit matrices of appropriate dimensions.\
The scipy function **solve_continuous_are** numerically solves the *Ricatti equation*.\
Controllability means, I think, that any starting configuration of the system may be moved to another predetermined position via the controls. Intuitively clear, that with this pendulum this is only the case for n = 1\
All of this is in books about (linear) control theory. 

In [ ]:
start4 = time.time()

pL = [g, m, l, r1, iXX, iYY, iZZ, c1, Fx, Fz] + Torque
pL_vals = [9.8, 1., 10., 5., 1, 2, 3, 1.,  0, 0] + [0. for i in range(len(Torque))]

A_lam = sm.lambdify(pL, F_A, cse=True)
B_lam = sm.lambdify(pL, F_B, cse=True)

A = A_lam(*pL_vals)
B = B_lam(*pL_vals)
print('determinant of A', det(A))
#Is the system controllable? This seems to be only a sufficient condition (?)
def controllable(a, b):
    a = np.matrix(a)
    b = np.matrix(b)
    n = a.shape[0]
    controllable_matrix = []
    for i in range(n):
        controllable_matrix.append(a ** i * b)
    controllable_matrix = np.hstack(controllable_matrix)
    return np.linalg.matrix_rank(controllable_matrix) == n
print('System controllable?  ', controllable(A, B))

#compute optimal gains K
Q = np.eye(A.shape[1])
R = np.eye(B.shape[1])
S = solve_continuous_are(A, B, Q, R) #solves Riccati equation
K = np.dot(np.dot(np.linalg.inv(R), B.T), S)

print('it took {:.3f} sec to calculate the feed back'.format(time.time() - start4))

**Numerical integration** of the **full system** using the optimum controls\
The optimal feed back is calculated each step of the integration.\
This is sensitive to the initial conditions, the disturbances from the equilibrium. If they are small enough, it runs very fast, otherwise, the integration runs 'forever'. (In these cases I always interrupted)
 

In [ ]:
start5 = time.time()

intervall = 20
schritte = 100

# The controls, which originally were in pL_vals will now, of course be the optimal feed back, and added during each step of the integration
pL1_vals = [9.8, 1., 10., 5.,  1., 2., 3., 1]
equilibrium_point = np.zeros(len(qL))

# now: kraft = K @ (x(equilibrium) - x(t)). I store the forces and the times in lists, so that I can plot them later.
kraefte = []
zeiten  = []

#@profile
def gradient(t, x, args):
# calculate the control force
    kraft = np.dot(K, equilibrium_point - x)
    kraefte.append(kraft)
    zeiten.append(t)
    arguments = np.hstack((x, args, kraft))
    return  np.array(np.linalg.solve(MM_lam(*arguments), force_lam(*arguments))).T[0]

# The initial conditions introduce some disturbances from the equilibrium point
aa = []
for i in range(int(len(u)/3)):
    aa.append(0.0 * np.random.choice((-1., 1.)))
    aa.append(0.15 * np.random.choice((-1., 1.)))
    aa.append(0.)
y0 = [0., 0.] + [0.15, 0., 0.15] * int(len(q)/3) + [0., 0.] + aa 
print('y0', y0)
y_store = list(y0)                                                      # store the initial conditions, so I do the solution using matrix exponentials below with the same initial conditions

times = np.linspace(0., intervall, int(schritte*intervall))
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL1_vals,), atol=1.e-10, rtol=1.e-10) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start5):.3f} CPU - sec  ')
#profiler.print_stats()

Plot the results\
It can easily be seen, that the system is driven to its equilibrium point, if the initial disturbances are not too great.\
Note, that the results are a bit different from those obtained by solving the linearized EOMs below. I do not know why.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

kraefte = np.array(kraefte)
for i, j in enumerate(('Fx', 'Fy', *['Torque on body ' + str(kk) for kk in range(1, n+1)])):
    ax2.plot(zeiten, kraefte[:, i], label=j)
ax2.set_title('Control forces / torques')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N] / Torque [Nm]')   
ax2.legend();

**Numerically integrate** the **linearized system**.

The sytem here is: $\dfrac{d}{dt} \bar y = F_A \cdot \bar y + F_B \cdot \bar u$, where $\bar u$ is the control.

Form the matrix $RHS$ to be calculated
 as follows:

First, form new state vektor $\hat y = \left[ \bar y, \bar u \right]$

The new rhs of the system is $RHS = \begin{bmatrix} F_A & F_B \\ 0_1 & 0_2 \end{bmatrix}$, where $0_i = \begin{bmatrix} 0. &....& 0. \\. &...&.\\ 0. &...& 0. \end{bmatrix}$ of appropriate dimensions.

Now I integrate the system numerically. Contrary to what I expected, this is *slower* than integration the full system. Apparently enlarging the state space from $\R^{|generalized \hspace{3pt} coordinates|}$ to $\R^{|generalized \hspace{3pt} coordinates| + |controls|}$ more than offsets the simpler force term.


In [ ]:
start6 = time.time()

RHS = sm.Matrix.hstack(F_A, F_B)
RHS = sm.Matrix.vstack(RHS, sm.zeros(F_B.shape[1], RHS.shape[1]))
print('RHS shape', RHS.shape)
print('RHS DS', me.find_dynamicsymbols(RHS))
print('RHS FS', RHS.free_symbols)
print(f'RHS contains {sm.count_ops(RHS):,} operations, {sm.count_ops(sm.cse(RHS)):,} after cse \n')

pL1 = [g, m, l, r1, iXX, iYY, iZZ, c1]
p1L_vals = [9.8, 1., 10., 5., 1, 2, 3, 1.]

# convert RHS from a sympy matrix to a numpy array
RHS_lam = sm.lambdify(pL1, RHS, cse=True)
RHS = RHS_lam(*pL1_vals)

# extend the equilibrium point to include the control forces
equilibrium_point1 = np.hstack((equilibrium_point, np.zeros(B.shape[1])))

# form y_hat, the initial conditions for the solution of the linearized system
y0 = np.array(list(y_store) + [0. for _ in range(B.shape[1])])      

# solve the linearized system, using matrix exponentials
resultat = []
kraefte  = []
zeiten   = [] 

#@profile
def gradient(t, y):
# get the optium control
    kraft = K @ (equilibrium_point - y[: len(qL)])
    kraefte.append(kraft)
    zeiten.append(t)
    y[len(qL):] = kraft
#    y = list(y[: len(qL)]) + list(kraft)
#    y = np.array(y)

    return RHS @ (y - equilibrium_point1)

times = np.linspace(0., intervall, int(schritte*intervall))
t_span = (0., intervall)

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, atol=1.e-10, rtol=1.e-10) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start6):.3f} CPU - sec  ')
#profiler.print_stats()

Plot the results

In [ ]:
resultat_hilfs = []
for i in range(len(times)):
    resultat_hilfs.append(resultat[i, : len(qL)])
resultat = np.array(resultat_hilfs)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

kraefte = np.array(kraefte)
for i, j in enumerate(('Fx', 'Fy', *['Torque on body ' + str(kk) for kk in range(1, n+1)])):
    ax2.plot(zeiten, kraefte[:, i], label=j)
ax2.set_title('Control forces / torques')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N] / Torque [Nm]')   
ax2.legend();

Here I use scipy's **matrix exponential** solver *expm(..)* to solve the differential equation without integrating.\
I use the fact, that this idfferential equation $\dfrac{d}{dt} \hat y = RHS \cdot \hat y$, with RHS = const., has the solution $\hat y = e^{\displaystyle{RHS \cdot t}} \cdot \hat y_0$\
I have to to it in steps, using the result of the previous step as input for the next step, otherwise of course the controls will have no effect.\
At least with this example, this is the *fastest* way of doing it.

In [ ]:
start7 = time.time()
y = np.array(list(y_store) + [0. for _ in range(B.shape[1])])      
# solve the linearized system, using matrix exponentials
resultat = []
kraefte  = []
zeiten   = [] 

for i in range(len(times)-1):
# get the optium control
    kraft = K @ (equilibrium_point - y[: len(qL)])
    kraefte.append(kraft)
    zeiten.append(times[i])
    y[len(qL):] = kraft
# Here I do the stepwise solution.
    y = expm(RHS * (times[i+1] - times[i])) @ y
    resultat.append((y))
    
print(f'to calculate {intervall:.3f} sec took {(time.time() - start7):.3f} CPU - sec  ')
print('are there nan values in the result?', np.isnan(resultat).any())

Plot the results

In [ ]:
resultat_hilfs = []
resultat = np.array(resultat)
for i in range(len(times)-1):
    resultat_hilfs.append(resultat[i, : len(qL)])
resultat = np.array(resultat_hilfs)
times = times[:-1]
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
for i in range(2, int(resultat.shape[1]/2)):
    wahl  = (i+1) % 3
    wahl1 = (i+1) // 3
    ax1.plot(times, np.rad2deg(resultat[:, i]), label=f'{bez[wahl]} - gen. angle of body {wahl1}')
ax1.set_title('Generalized angles')
ax1.set_ylabel('Angle in degree [°]')
ax1.legend();

kraefte = np.array(kraefte)
for i, j in enumerate(('Fx', 'Fy', *['Torque on body ' + str(kk) for kk in range(1, n+1)])):
    ax2.plot(zeiten, kraefte[:, i], label=j)
ax2.set_title('Control forces / torques')
ax2.set_xlabel('Time [s]')
ax2.set_ylabel('Force [N] / Torque [Nm]')   
ax2.legend();
print(f'running the complete program takes {(time.time() - start):.3f} CPU - sec')